In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.api.types import is_string_dtype
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv('C:/Users/85059/Desktop/DATA7703 Machine Learning/house-prices-advanced-regression-techniques/train.csv')
df.shape[1]
print(df['SalePrice'].describe())
df_clean = df.copy()
miss_col = df_clean.columns[df.isnull().any()].tolist()

df_clean = df.copy()
basement_na_list = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
fireplace_na_liat = ['FireplaceQu']
garage_na_list = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
pool_na_list = ['PoolQC']
fence_na_list = ['Fence']
misfeature_na_list = ['MiscFeature']

# 把缺失的值简单处理下
for missing in miss_col:
    if missing == 'Alley':
        df_clean['Alley'] = df_clean['Alley'].replace(np.nan,'No alley access')
    elif missing in basement_na_list:
        df_clean.loc[:,basement_na_list] = df_clean.loc[:,basement_na_list].replace(np.nan,'No basement')
    elif missing in fireplace_na_liat:
        df_clean.loc[:,fireplace_na_liat] = df_clean.loc[:,fireplace_na_liat].replace(np.nan,'No fireplace')
    elif missing in garage_na_list:
        df_clean.loc[:,garage_na_list] = df_clean.loc[:,garage_na_list].replace(np.nan,'No garage')
    elif missing in pool_na_list:
        df_clean.loc[:,pool_na_list] = df_clean.loc[:,pool_na_list].replace(np.nan,'No pool')
    elif missing in fence_na_list:
        df_clean.loc[:,fence_na_list] = df_clean.loc[:,fence_na_list].replace(np.nan,'No fence')
    elif missing in misfeature_na_list:
        df_clean.loc[:,misfeature_na_list] = df_clean.loc[:,misfeature_na_list].replace(np.nan,'None')


df_clean.isnull().sum()
miss_col_clean = df_clean.columns[df_clean.isnull().any()].tolist()
print(miss_col_clean)
df_clean.loc[:, miss_col_clean].isnull().sum()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64
['LotFrontage', 'MasVnrType', 'MasVnrArea', 'Electrical', 'GarageYrBlt']


LotFrontage    259
MasVnrType       8
MasVnrArea       8
Electrical       1
GarageYrBlt     81
dtype: int64

In [5]:
# 找出哪几列是numerical data，哪几列是categorical data
def find_index_num_str(df):
    is_string = []
    for index in range(df.shape[1]):
        data = df.iloc[:,index]
        is_string.append(is_string_dtype(data))
    # print(is_string)

    index_numerical = [i for i, istring in enumerate(is_string) if istring == False]
    # print(index_false)
    index_string = [i for i, istring in enumerate(is_string) if istring == True]
    
    return index_numerical, index_string

print(miss_col_clean)

['LotFrontage', 'MasVnrType', 'MasVnrArea', 'Electrical', 'GarageYrBlt']


In [13]:
# encoding
i_num, i_str = find_index_num_str(df_clean)
print(i_str)
df_enc = df_clean.copy()
for col in i_str:
    labelencoder = LabelEncoder()
    df_enc.iloc[:,col] = labelencoder.fit_transform(df_enc.iloc[:,col])
df_enc_x = df_enc.loc[:, df_enc.columns != 'SalePrice']

df_enc_x = df_enc_x.drop(['Id'], axis=1)
df_enc_x

[2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 35, 39, 40, 41, 42, 53, 55, 57, 58, 60, 63, 64, 65, 72, 73, 74, 78, 79]


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,3,65.0,8450,1,1,3,3,0,4,...,0,0,3,4,1,0,2,2008,8,4
1,20,3,80.0,9600,1,1,3,3,0,2,...,0,0,3,4,1,0,5,2007,8,4
2,60,3,68.0,11250,1,1,0,3,0,4,...,0,0,3,4,1,0,9,2008,8,4
3,70,3,60.0,9550,1,1,0,3,0,0,...,0,0,3,4,1,0,2,2006,8,0
4,60,3,84.0,14260,1,1,0,3,0,2,...,0,0,3,4,1,0,12,2008,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,62.0,7917,1,1,3,3,0,4,...,0,0,3,4,1,0,8,2007,8,4
1456,20,3,85.0,13175,1,1,3,3,0,4,...,0,0,3,2,1,0,2,2010,8,4
1457,70,3,66.0,9042,1,1,3,3,0,4,...,0,0,3,0,3,2500,5,2010,8,4
1458,20,3,68.0,9717,1,1,3,3,0,4,...,0,0,3,4,1,0,4,2010,8,4


In [7]:
# dfreg = df_enc_x[df_enc_x['A'].notna()]
# dfimp = df_enc_x[df_enc_x['A'].isna()]

# from sklearn.neural_network import MLPRegressor    
# regr = MLPRegressor(random_state=1, max_iter=200).fit(dfreg['B'].values.reshape(-1, 1), dfreg['A'])
# regr.score(dfreg['B'].values.reshape(-1, 1), dfreg['A'])

# regr.predict(dfimp['B'].values.reshape(-1, 1))

In [8]:
def random_imputation(df, feature):

    number_missing = df[feature].isnull().sum()
    observed_values = df.loc[df[feature].notnull(), feature]
    df.loc[df[feature].isnull(), feature + '_imp'] = np.random.choice(observed_values, number_missing, replace = True)
    
    return df
# x_enc.loc[:, miss_col_clean].isnull().sum()
dataframe_x = df_enc_x.copy()

for feature in miss_col_clean:
    dataframe_x[feature + '_imp'] = dataframe_x[feature]
    df_randomimp = random_imputation(dataframe_x, feature)
    
print(df_randomimp)

      MSSubClass  MSZoning  LotFrontage  LotArea  Street  Alley  LotShape  \
0             60         3         65.0     8450       1      1         3   
1             20         3         80.0     9600       1      1         3   
2             60         3         68.0    11250       1      1         0   
3             70         3         60.0     9550       1      1         0   
4             60         3         84.0    14260       1      1         0   
...          ...       ...          ...      ...     ...    ...       ...   
1455          60         3         62.0     7917       1      1         3   
1456          20         3         85.0    13175       1      1         3   
1457          70         3         66.0     9042       1      1         3   
1458          20         3         68.0     9717       1      1         3   
1459          20         3         75.0     9937       1      1         3   

      LandContour  Utilities  LotConfig  ...  MiscVal  MoSold  YrSold  \
0 

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_imp,MasVnrType_imp,MasVnrArea_imp,Electrical_imp,GarageYrBlt_imp
0,60,3,65.0,8450,1,1,3,3,0,4,...,0,2,2008,8,4,65.0,1,196.0,4,2003.0
1,20,3,80.0,9600,1,1,3,3,0,2,...,0,5,2007,8,4,80.0,2,0.0,4,1976.0
2,60,3,68.0,11250,1,1,0,3,0,4,...,0,9,2008,8,4,68.0,1,162.0,4,2001.0
3,70,3,60.0,9550,1,1,0,3,0,0,...,0,2,2006,8,0,60.0,2,0.0,4,1998.0
4,60,3,84.0,14260,1,1,0,3,0,2,...,0,12,2008,8,4,84.0,1,350.0,4,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,62.0,7917,1,1,3,3,0,4,...,0,8,2007,8,4,62.0,2,0.0,4,1999.0
1456,20,3,85.0,13175,1,1,3,3,0,4,...,0,2,2010,8,4,85.0,3,119.0,4,1978.0
1457,70,3,66.0,9042,1,1,3,3,0,4,...,2500,5,2010,8,4,66.0,2,0.0,4,1941.0
1458,20,3,68.0,9717,1,1,3,3,0,4,...,0,4,2010,8,4,68.0,2,0.0,0,1950.0


In [12]:
# 线性imputation的开始
from sklearn.linear_model import LinearRegression

deter_data = pd.DataFrame(columns = ["Det" + name for name in miss_col_clean])
dataframe_x
# after random imputation, predict data in regression 

for feature in miss_col_clean:
        
    deter_data["Det" + feature] = dataframe_x[feature + "_imp"]
    parameters = list(set(dataframe_x.columns) - set(miss_col_clean) - {feature + '_imp'})
    
    #Create a Linear Regression model to estimate the missing data
    model = LinearRegression()
    model.fit(X = dataframe_x[parameters], y = dataframe_x[feature + '_imp'])
    
    #observe that I preserve the index of the missing data from the original dataframe
    deter_data.loc[dataframe_x[feature].isnull(), "Det" + feature] = model.predict(dataframe_x[parameters])[dataframe_x[feature].isnull()]
         
dataframe_x

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,MiscVal,MoSold,YrSold,SaleType,SaleCondition,LotFrontage_imp,MasVnrType_imp,MasVnrArea_imp,Electrical_imp,GarageYrBlt_imp
0,60,3,65.0,8450,1,1,3,3,0,4,...,0,2,2008,8,4,65.0,1,196.0,4,2003.0
1,20,3,80.0,9600,1,1,3,3,0,2,...,0,5,2007,8,4,80.0,2,0.0,4,1976.0
2,60,3,68.0,11250,1,1,0,3,0,4,...,0,9,2008,8,4,68.0,1,162.0,4,2001.0
3,70,3,60.0,9550,1,1,0,3,0,0,...,0,2,2006,8,0,60.0,2,0.0,4,1998.0
4,60,3,84.0,14260,1,1,0,3,0,2,...,0,12,2008,8,4,84.0,1,350.0,4,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,62.0,7917,1,1,3,3,0,4,...,0,8,2007,8,4,62.0,2,0.0,4,1999.0
1456,20,3,85.0,13175,1,1,3,3,0,4,...,0,2,2010,8,4,85.0,3,119.0,4,1978.0
1457,70,3,66.0,9042,1,1,3,3,0,4,...,2500,5,2010,8,4,66.0,2,0.0,4,1941.0
1458,20,3,68.0,9717,1,1,3,3,0,4,...,0,4,2010,8,4,68.0,2,0.0,0,1950.0


In [9]:
imp_col = [feature + '_imp' for feature in miss_col_clean]
dict_rename = dict(zip(imp_col, miss_col_clean))
dict_rename
df_reg_imp = dataframe_x.drop(miss_col_clean, axis=1)
    
df_reg_imp.rename(columns = dict_rename, inplace = True)
df_reg_imp.isnull().sum()
df_reg_imp = pd.concat([df_reg_imp, df_clean['SalePrice']], axis=1)

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold, cross_val_score

def cv_decision_tree(x, y, kfold):
    clf = DecisionTreeClassifier()
    k_folds = KFold(n_splits = kfold)
    scores = cross_val_score(clf, x, y, cv = k_folds)
    # print(scores)
#     print('Average CV: ',scores.mean())
    return scores.mean()

print('Avr. CV of linear regression imputation: ',cv_decision_tree(df_reg_imp, df_clean['SalePrice'], 10))

Avr. CV of linear regression imputation:  0.43630136986301365


In [11]:
# df_reg_imp.to_csv('df_reg_imp.csv', encoding='utf-8', index=False)

PermissionError: [Errno 13] Permission denied: 'df_reg_imp.csv'